# Imports

In [2]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, RYGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from typing import Dict, Tuple, List
import numpy as np
from numpy import linalg as LA
from qiskit.opflow import X, Z, I, H
from qiskit.quantum_info import Pauli
import matplotlib
from qiskit import assemble, Aer
from qiskit import IBMQ
from qiskit import *
from qiskit.visualization import plot_histogram
import math
from qiskit.algorithms.optimizers import L_BFGS_B, COBYLA

# Implementing the ansatz from qiskit tutorial

In [3]:
def calc_linear_entangelment_ansatz(num_of_qubits, thetas, depth):
    if thetas.size != 2*num_of_qubits*(depth + 1):
        print("ERROR with thetas size")
    else:
        qr = QuantumRegister(num_of_qubits, name="qubit")
        qc = QuantumCircuit(qr)
        
        for iteration in range(depth):
            for qubit_index in range(num_of_qubits):
                RY_theta_index = iteration*2*num_of_qubits + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
            for qubit_index in range(num_of_qubits - 1):
                qc.append(CXGate(), [qr[qubit_index], qr[qubit_index + 1]])
        
        for qubit_index in range(num_of_qubits):
                RY_theta_index = 2*num_of_qubits*depth + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
        
        return qc

def calc_full_entangelment_ansatz(num_of_qubits, thetas, depth):
    if thetas.size != 2*num_of_qubits*(depth + 1):
        print("ERROR with thetas size")
    else:
        qr = QuantumRegister(num_of_qubits, name="qubit")
        qc = QuantumCircuit(qr)
        
        for iteration in range(depth):
            for qubit_index in range(num_of_qubits):
                RY_theta_index = iteration*2*num_of_qubits + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
            for qubit_index in range(num_of_qubits - 1):
                for target_qubit_index in range(qubit_index + 1, num_of_qubits):
                    qc.append(CXGate(), [qr[qubit_index], qr[target_qubit_index]])
        
        for qubit_index in range(num_of_qubits):
                RY_theta_index = 2*num_of_qubits*depth + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
        
        return qc

def calc_ansatz_circ(num_of_qubits, entangelment, thetas, depth):
    if entangelment == "linear":
        return calc_linear_entangelment_ansatz(num_of_qubits, thetas, depth)
    assert entangelment == "full", "entangelment has to be linear or full"
    return calc_full_entangelment_ansatz(num_of_qubits, thetas, depth)

# Generating the hamiltonians

In [4]:
H2_molecule_Hamiltonian = -0.5053051899926562*(I^I) + \
                            -0.3277380754984016*(Z^I) + \
                            0.15567463610622564*(Z^Z) + \
                            -0.3277380754984016*(I^Z)

def create_pauli_string_with_pauli_op_on_index_i(pauli_op, i, qubits_num):
    if i == 1:
        pauli_string = pauli_op
        for qubit in range(qubits_num - 1):
            pauli_string = pauli_string ^ I
        return pauli_string
    
    pauli_string = I
    for qubit in range(2, qubits_num + 1):
        if qubit == i:
            pauli_string = pauli_string ^ pauli_op
        else:
            pauli_string = pauli_string ^ I
            
    return pauli_string

def create_pauli_string_with_pauli_ops_on_index_i_and_j(pauli_op_second, i, pauli_op_first, j, qubits_num):
    if j == 1:
        pauli_string = pauli_op_first
        for qubit in range(2, qubits_num + 1):
            if qubit == i:
                pauli_string = pauli_string ^ pauli_op_second
            else:
                pauli_string = pauli_string ^ I
        return pauli_string
    
    pauli_string = I
    for qubit in range(2, qubits_num + 1):
        if qubit == j:
            pauli_string = pauli_string ^ pauli_op_first
        elif qubit == i:
            pauli_string = pauli_string ^ pauli_op_second
        else:
            pauli_string = pauli_string ^ I
            
    return pauli_string
    
def get_Ising_model_hamiltonian():
    hamiltonian = I
    for qubit in range(QUBITS_NUM - 1):
        hamiltonian = hamiltonian^I
    hamiltonian = 0 * hamiltonian
    
    for i in range(1, QUBITS_NUM + 1):
        a_i = np.random.random_sample()
        x_i = create_pauli_string_with_pauli_op_on_index_i(X, i, QUBITS_NUM)
        hamiltonian = hamiltonian + a_i * x_i
        for j in range(1, i):
            J_ij = np.random.random_sample()
            z_ij = create_pauli_string_with_pauli_ops_on_index_i_and_j(Z, i, Z, j, QUBITS_NUM)
            hamiltonian = hamiltonian + J_ij * z_ij
    
    return hamiltonian
    
def get_hamiltonian(hamiltonian_type):
    if hamiltonian_type == "Ising Model":
        return get_Ising_model_hamiltonian()
    assert hamiltonian_type == "Molecular", "hamiltonian_type has to be Ising Model or Molecular"
    return H2_molecule_Hamiltonian

# Expectation Value

In [5]:
reducing_to_pauli_z_dict = {
    Pauli('I'): Pauli('I'),
    Pauli('Z'): Pauli('Z'),
    Pauli('X'): Pauli('Z'),
    Pauli('Y'): Pauli('Z')
} 

def transfrom_hamiltonian_into_pauli_string(hamiltonian):
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    pauli_coeff = list(map(lambda pauli_operator: pauli_operator.coeff, pauli_operators))
    return (pauli_strings, pauli_coeff)

def reduce_pauli_matrixes_into_sigma_z(pauli_string):
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = pauli_string[matrix_index]
        pauli_string[matrix_index].insert(reducing_to_pauli_z_dict[pauli_matrix])
    
    print(pauli_string)
    return pauli_string

def get_z_reduction_for_pauli_string(qc, pauli_string):
    qr = QuantumRegister(QUBITS_NUM, name="q")
    exdend_qc = QuantumCircuit(qr)
    pauli_string = str(pauli_string)
    for qubit_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            exdend_qc.append(HGate(), [qr[qubit_index]])
        elif pauli_matrix == "Y":
            exdend_qc.append(HGate(), [qr[qubit_index]])
            exdend_qc.append(SGate(), [qr[qubit_index]])
    qc = qc.compose(exdend_qc)
    return qc

def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()

    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, BACKEND) 
#     Qobj = assemble(transpiled_quantum_state_circuit)
    result = BACKEND.run(transpiled_quantum_state_circuit).result() 
    counts = result.get_counts(transpiled_quantum_state_circuit)
    
    return get_probability_distribution(counts)

def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, probas_distribution):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

def get_expectation_value(pauli_string, probas_distribution):
        return calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                pauli_string, probas_distribution)

# Calculating the target function

In [6]:
def target_function(thetas):
    ansatz = calc_ansatz_circ(QUBITS_NUM, Entangelment, thetas, Depth)
    
    qr = QuantumRegister(QUBITS_NUM, name="qubit")
    qc = QuantumCircuit(qr)
    qc.initialize(Init_state, qc.qubits)
    
    total_expectation_value = 0
    pauli_strings, pauli_coeffs = transfrom_hamiltonian_into_pauli_string(Ham)
    for pauli_index, pauli_string in enumerate(pauli_strings):
        qc_per_pauli_string = qc.compose(ansatz)
        qc_per_pauli_string = get_z_reduction_for_pauli_string(qc_per_pauli_string, pauli_string)
        probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(qc_per_pauli_string)
        total_expectation_value += pauli_coeffs[pauli_index] * get_expectation_value(pauli_string, probas_distribution)
    print("total expectation value:")
    print(total_expectation_value)
    return total_expectation_value
        

# Implementing the VQE algorithem

In [7]:
def VQE(optimazer, initial_thetas):
    point, value, nfev = optimazer.optimize(QUBITS_NUM*2*(Depth + 1), target_function, initial_point=initial_thetas)
    base_energy = value
    print("The VQE result for the minimal energy of the Hamiltonian given is:")
    print(base_energy)

# Setting the simulator and optimazer

In [9]:
bfgs_optimizer = L_BFGS_B(maxiter=2)
cobyla_optimazer = COBYLA(maxiter=50)

provider = IBMQ.enable_account('4cd532424f249f20233857b3b211eb28dfc0e790386bd2ea14d3e0d03c867dfcfec4c2c968e4693f1c9caf7b3f6fad3f6a6393065fb45719692fd1a5177536cb')
real_backend = provider.get_backend('ibmq_belem')
simulator_backend = Aer.get_backend('aer_simulator') 

# VQE with Transverse Ising Model Hamiltonian + LINEAR Entangelment Ansatz

In [8]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)
    

total expectation value:
1.627374133563496
total expectation value:
1.7268951033599416
total expectation value:
1.617097749901638
total expectation value:
1.6369024346248398
total expectation value:
1.567207675995089
total expectation value:
1.5883579080012016
total expectation value:
1.6633591504264462
total expectation value:
1.6150809032929387
total expectation value:
1.5806960084556805
total expectation value:
1.6004157055768398
total expectation value:
1.668288062326499
total expectation value:
1.612349463801928
total expectation value:
1.6122417284793378
total expectation value:
1.6486773548690177
total expectation value:
1.6108940308524229
total expectation value:
1.6109925771760356
total expectation value:
1.5864592760014262
total expectation value:
1.604185121176648
total expectation value:
1.6013961085714792
total expectation value:
1.6007367727238186
total expectation value:
1.6087708214480836
total expectation value:
1.6115926086536942
total expectation value:
1.61642283021

total expectation value:
1.642938498966407
total expectation value:
1.6743624684628717
total expectation value:
1.6438238169915227
total expectation value:
1.620316228243507
total expectation value:
1.6465900655091679
total expectation value:
1.574421190038819
total expectation value:
1.622612741109412
total expectation value:
1.6044529094113196
total expectation value:
1.617374822441798
total expectation value:
1.6056861409765784
total expectation value:
1.6019903987681554
total expectation value:
1.6445883890039694
total expectation value:
1.5860972875357828
total expectation value:
1.59941984894833
total expectation value:
1.6314509860371387
total expectation value:
1.5302750960349374
total expectation value:
1.643402872536809
total expectation value:
1.5919535187476272
total expectation value:
1.5688039797527926
total expectation value:
1.6053209371423574
total expectation value:
1.5977290988916542
total expectation value:
1.6184682301516289
total expectation value:
1.7121582548210

total expectation value:
1.610549539301922
total expectation value:
1.5670684760583615
total expectation value:
1.5971327783542475
total expectation value:
1.674097008488402
total expectation value:
1.6066660060108764
total expectation value:
1.5884916395410231
total expectation value:
1.5621941781076452
total expectation value:
1.591547216168049
total expectation value:
1.6080536252506357
total expectation value:
1.5893786861021024
total expectation value:
1.6398017035314374
total expectation value:
1.6733608460740714
total expectation value:
1.6001603371698925
total expectation value:
1.6374551969843902
total expectation value:
1.6067351830379242
total expectation value:
1.6847991556185768
total expectation value:
1.611850344987245
total expectation value:
1.6020348782780052
total expectation value:
1.6167690380863597
total expectation value:
1.6699048736367281
total expectation value:
1.6584175494189184
total expectation value:
1.58381694609391
total expectation value:
1.65509808667

total expectation value:
1.5968601078802935
total expectation value:
1.5592303186376189
total expectation value:
1.5970184489298354
total expectation value:
1.617217240944122
total expectation value:
1.6165747683083065
total expectation value:
1.6198321859535558
total expectation value:
1.7301369243399216
total expectation value:
1.6408000389623059
total expectation value:
1.6753313895601596
total expectation value:
1.685657683931323
total expectation value:
1.6019294686922356
total expectation value:
1.6768891721139503
total expectation value:
1.7028321354088862
total expectation value:
1.6329339155229068
total expectation value:
1.7024815920733594
total expectation value:
1.5562849369810383
total expectation value:
1.6145712995029387
total expectation value:
1.5243906481045086
total expectation value:
1.6689760501646316
total expectation value:
1.5551519837792633
total expectation value:
1.5391797037628345
total expectation value:
1.6025055355443645
total expectation value:
1.5399120

total expectation value:
1.6070215393979752
total expectation value:
1.6410007041656365
total expectation value:
1.6362856733245579
total expectation value:
1.5563959219169092
total expectation value:
1.6173919574351543
total expectation value:
1.6303519687174086
total expectation value:
1.6746166882478537
total expectation value:
1.6540502825262662
total expectation value:
1.6559309083246225
total expectation value:
1.6280400064705727
total expectation value:
1.5765283711773623
total expectation value:
1.6019347873990113
total expectation value:
1.5923537731442137
total expectation value:
1.6755908443919831
total expectation value:
1.590405981335993
total expectation value:
1.6584279095234784
total expectation value:
1.5670725537167693
total expectation value:
1.6097009128903907
total expectation value:
1.6040280217124163
total expectation value:
1.6444981723536418
total expectation value:
1.618038007927015
total expectation value:
1.6121874604049982
total expectation value:
1.6020683

# VQE with Transverse Ising Model Hamiltonian + FULL Entangelment Ansatz

In [9]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

total expectation value:
1.3742652408936404
total expectation value:
1.3981197261968035
total expectation value:
1.3743024888554183
total expectation value:
1.390291904058989
total expectation value:
1.3354823267982523
total expectation value:
1.4237203892966226
total expectation value:
1.4676204743583214
total expectation value:
1.4471663289324166
total expectation value:
1.4032535749022244
total expectation value:
1.4131359331892988
total expectation value:
1.3467792915967594
total expectation value:
1.401516278550093
total expectation value:
1.4411796528165275
total expectation value:
1.4087779233364957
total expectation value:
1.3708416886645514
total expectation value:
1.4749411643027637
total expectation value:
1.4023753200401674
total expectation value:
1.418764542025224
total expectation value:
1.3746391907715794
total expectation value:
1.3749952720604384
total expectation value:
1.3953936130516793
total expectation value:
1.373365617810342
total expectation value:
1.489013074

total expectation value:
1.3837905520083011
total expectation value:
1.4461003775161303
total expectation value:
1.3719656926931123
total expectation value:
1.3709527164058004
total expectation value:
1.3675331409569882
total expectation value:
1.3470221559544617
total expectation value:
1.4119516775277259
total expectation value:
1.3655901235512558
total expectation value:
1.4023639581408671
total expectation value:
1.390665572745787
total expectation value:
1.4132666554602227
total expectation value:
1.362901439624069
total expectation value:
1.3700241893009684
total expectation value:
1.3855474433851167
total expectation value:
1.3176112010993852
total expectation value:
1.3925684297992154
total expectation value:
1.4095124709588105
total expectation value:
1.4262186079641241
total expectation value:
1.3723537603675608
total expectation value:
1.3100715703328567
total expectation value:
1.4135101626017426
total expectation value:
1.3665214519304656
total expectation value:
1.3905603

total expectation value:
1.3243627819001047
total expectation value:
1.3268519626589543
total expectation value:
1.3214630001349645
total expectation value:
1.3144694792551193
total expectation value:
1.357907222081591
total expectation value:
1.3279282131226067
total expectation value:
1.474419096917217
total expectation value:
1.4659119033886796
total expectation value:
1.3317446059267573
total expectation value:
1.4042885647181123
total expectation value:
1.4193732331741602
total expectation value:
1.3299756652684405
total expectation value:
1.4807973559956598
total expectation value:
1.4142927309063438
total expectation value:
1.381512030143041
total expectation value:
1.3844224957441964
total expectation value:
1.4631799708781748
total expectation value:
1.3802237558131916
total expectation value:
1.3149915942945622
total expectation value:
1.3827822882069647
total expectation value:
1.3965603706967133
total expectation value:
1.4079332289919404
total expectation value:
1.30239011

total expectation value:
1.4818670213189362
total expectation value:
1.3508522454326428
total expectation value:
1.4702196575795305
total expectation value:
1.4010290816217372
total expectation value:
1.31267826024781
total expectation value:
1.292035542473591
total expectation value:
1.459318744269681
total expectation value:
1.3843009923145217
total expectation value:
1.4055907403956756
total expectation value:
1.407192249494737
total expectation value:
1.3574107154131951
total expectation value:
1.3098091867367747
total expectation value:
1.3655429468224425
total expectation value:
1.4359140902553256
total expectation value:
1.3716749478311092
total expectation value:
1.4153713338200955
total expectation value:
1.4090636014932263
total expectation value:
1.3922928092663125
total expectation value:
1.379655857218752
total expectation value:
1.3686627148557027
total expectation value:
1.3717796742350266
total expectation value:
1.4680554976003035
total expectation value:
1.43151191462

total expectation value:
1.3932567810226397
total expectation value:
1.4165528471884992
total expectation value:
1.4141256453715028
total expectation value:
1.343817457634042
total expectation value:
1.3329957775899173
total expectation value:
1.3965579468851241
total expectation value:
1.4196076773844062
total expectation value:
1.4268223283835046
total expectation value:
1.4256170246630466
total expectation value:
1.4273303988908483
total expectation value:
1.4452842653435511
total expectation value:
1.4057844350595643
total expectation value:
1.3266100302498194
total expectation value:
1.4190369557316695
total expectation value:
1.3876177318801362
total expectation value:
1.3336227983944557
total expectation value:
1.3921398640061562
total expectation value:
1.4214666778432758
total expectation value:
1.4516519079613692
total expectation value:
1.3672775316007588
total expectation value:
1.312634158121054
total expectation value:
1.3464005725377894
total expectation value:
1.4249395

# VQE with H2 Molecule Hamiltonian + LINEAR Entangelment Ansatz

In [10]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

total expectation value:
-0.2973482378166863
total expectation value:
-0.3243290299666682
total expectation value:
-0.274512252462246
total expectation value:
-0.31335114773403133
total expectation value:
-0.2874425585620737
total expectation value:
-0.3286018000882315
total expectation value:
-0.29731630263491804
total expectation value:
-0.29603607577750246
total expectation value:
-0.2999567427034692
total expectation value:
-0.3476291884514926
total expectation value:
-0.2896029321090064
total expectation value:
-0.2853462037779768
total expectation value:
-0.2838099018692182
total expectation value:
-0.30330138916321353
total expectation value:
-0.3354670333060441
total expectation value:
-0.32620139328048964
total expectation value:
-0.305845801087511
total expectation value:
-0.2328088514519843
total expectation value:
-0.2526523677419266
total expectation value:
-0.20094725933279878
total expectation value:
-0.24801152683388258
total expectation value:
-0.26275017748469603
tota

total expectation value:
-0.30462959299293113
total expectation value:
-0.29254749840085215
total expectation value:
-0.3126150061610699
total expectation value:
-0.30866225565403566
total expectation value:
-0.31403923813504114
total expectation value:
-0.30310933287470626
total expectation value:
-0.3151754598759012
total expectation value:
-0.2945317832501617
total expectation value:
-0.3048535844632066
total expectation value:
-0.28758656367862934
total expectation value:
-0.29285147622484753
total expectation value:
-0.3138152466647657
total expectation value:
-0.3001007478200248
total expectation value:
-0.34066792708942645
total expectation value:
-0.32384896344504976
total expectation value:
-0.29822845870585296
total expectation value:
-0.312582996779652
total expectation value:
-0.3091103869938858
total expectation value:
-0.29746030775147364
total expectation value:
-0.31343113408775125
total expectation value:
-0.29499588218089606
total expectation value:
-0.322264684563989

# VQE with H2 Molecule Hamiltonian + FULL Entangelment Ansatz

In [11]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

total expectation value:
-0.32239272208966063
total expectation value:
-0.2836818643435467
total expectation value:
-0.320232348542728
total expectation value:
-0.2782889142716571
total expectation value:
-0.2986285388737515
total expectation value:
-0.2887067436289556
total expectation value:
-0.32640941715988103
total expectation value:
-0.3027412620882256
total expectation value:
-0.3072380236800644
total expectation value:
-0.2950119239714298
total expectation value:
-0.31383121425564975
total expectation value:
-0.3078621695178881
total expectation value:
-0.3004528510156214
total expectation value:
-0.2898269977789315
total expectation value:
-0.31530349740157265
total expectation value:
-0.30187708298959265
total expectation value:
-0.3092543921104415
total expectation value:
-0.4447666795065896
total expectation value:
-0.41810598117078646
total expectation value:
-0.4206664348856177
total expectation value:
-0.4386215460712048
total expectation value:
-0.4364772143148059
total

total expectation value:
-0.3039574701828055
total expectation value:
-0.29893259089739654
total expectation value:
-0.30738202879662
total expectation value:
-0.3010929644443292
total expectation value:
-0.28845074277726235
total expectation value:
-0.3148073890894204
total expectation value:
-0.3075581174942431
total expectation value:
-0.31679167393873003
total expectation value:
-0.3046455605838153
total expectation value:
-0.3200723016356386
total expectation value:
-0.3038934514199697
total expectation value:
-0.27792084348517637
total expectation value:
-0.2932516305923957
total expectation value:
-0.2955880186373019
total expectation value:
-0.3207444244457643
total expectation value:
-0.3158636244765606
total expectation value:
-0.30435747615105446
total expectation value:
-0.2793931266310992
total expectation value:
-0.2976523640399809
total expectation value:
-0.3110787042523113
total expectation value:
-0.28267368012835814
total expectation value:
-0.30381346506624984
total

total expectation value:
-0.3133030965620798
total expectation value:
-0.33036209346726564
total expectation value:
-0.3116228637364151
total expectation value:
-0.3259453924287963
total expectation value:
-0.312582996779652
total expectation value:
-0.3113027699222364
total expectation value:
-0.2926594941359899
total expectation value:
-0.2947078719477848
total expectation value:
-0.2960840527498045
total expectation value:
-0.2988365627531429
total expectation value:
-0.3130950726826884
total expectation value:
-0.30022878534569636
total expectation value:
-0.3411320260201608
total expectation value:
-0.2660787821539066
total expectation value:
-0.29489985403664243
total expectation value:
-0.31816792894039925
total expectation value:
-0.33789944949520384
total expectation value:
-0.300804954211218
total expectation value:
-0.29880455337172507
total expectation value:
-0.30086897297405374
total expectation value:
-0.30205309748756615
total expectation value:
-0.3060378573760183
tota

# VQE with Transverse Ising Model Hamiltonian + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [12]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)
    

total expectation value:
0.6168305419058646
total expectation value:
0.9273382375343067
total expectation value:
0.9574093258301368
total expectation value:
0.9230769530093901
total expectation value:
0.8384750106188845
total expectation value:
0.7852120968526689
total expectation value:
0.6851701087571249
total expectation value:
0.7847254871929557
total expectation value:
0.6065439050036993
total expectation value:
0.8646733597342509
total expectation value:
0.8018021090161921
total expectation value:
0.49893146068373984
total expectation value:
0.6160291837467828
total expectation value:
0.47746008502178994
total expectation value:
0.9407344198473648
total expectation value:
0.2341563012349541
total expectation value:
0.49427797424279585
total expectation value:
0.5338594376342312
total expectation value:
0.3906567509797385
total expectation value:
0.4022249720814854
total expectation value:
0.39342387477906254
total expectation value:
0.29678518496630507
total expectation value:
0.

# VQE with Transverse Ising Model Hamiltonian + FULL Entangelment Ansatz Using COBYLA Optimazer

In [13]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 4
Ham = get_hamiltonian("Ising Model")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

total expectation value:
1.6696222661536964
total expectation value:
2.053115172620362
total expectation value:
1.9869528973366766
total expectation value:
1.8155876077567243
total expectation value:
1.539358114972018
total expectation value:
1.4715366153451725
total expectation value:
1.4981831232551226
total expectation value:
1.4896233496519167
total expectation value:
1.3652450386137345
total expectation value:
1.5171466623943886
total expectation value:
1.06363092413652
total expectation value:
1.4663586869139589
total expectation value:
1.2792667832217768
total expectation value:
1.5054302662411372
total expectation value:
1.3421244457429649
total expectation value:
1.4758697895725572
total expectation value:
1.4226972195418195
total expectation value:
1.4445132681312025
total expectation value:
1.4914269332878305
total expectation value:
1.055530333819951
total expectation value:
1.2436947400086094
total expectation value:
1.2145917579610712
total expectation value:
1.4241448267

# VQE with H2 Molecule Hamiltonian + LINEAR Entangelment Ansatz Using COBYLA Optimazer

In [14]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "linear"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

total expectation value:
-0.2856662233925059
total expectation value:
-0.02888508133466633
total expectation value:
-0.3782422662960818
total expectation value:
-0.3646079022043601
total expectation value:
-0.6294065410114059
total expectation value:
-0.5440961082921396
total expectation value:
-0.5484964707414768
total expectation value:
-0.5681796433257311
total expectation value:
-0.5446075905979789
total expectation value:
-0.4890618758162538
total expectation value:
-0.7849544751856533
total expectation value:
-0.738177995318732
total expectation value:
-0.7885391549062067
total expectation value:
-0.8572080482724128
total expectation value:
-0.8231381798336421
total expectation value:
-0.8688420115246416
total expectation value:
-0.8885415226980282
total expectation value:
-0.7124464594398424
total expectation value:
-0.9427752980395183
total expectation value:
-0.9219716487063392
total expectation value:
-0.9179227958554016
total expectation value:
-0.9148183310557658
total expe

# VQE with H2 Molecule Hamiltonian + FULL Entangelment Ansatz Using COBYLA Optimazer

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = simulator_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian + FULL Entangelment Ansatz Using COBYLA Optimazer on REAL HARDWAWE

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024
BACKEND = real_backend

QUBITS_NUM = 2
Ham = get_hamiltonian("Molecular")
Entangelment = "full"
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = cobyla_optimazer
VQE(optimazer, initial_thetas)